# Mesa Tutorial

Welcome! There are a few slides accompanying this workshop. You can find them [here](https://docs.google.com/presentation/d/1HdxIk6XM6W-3IvTpLS_XOvetDfMACZKnFQlNKKXnMsA/edit?usp=sharing).

## Introduction

### Imports

Let's start by importing the packages we need.
* Note we must install Mesa, since Colab does not have it by default.
* If you are using your own computer, make sure you create a new environment. Install Mesa in your new environment and activate the environment.

In [ ]:
install_mesa = input("Do you want me to install mesa? [yes/no]")
if install_mesa.lower() == "yes":
  %pip install --quiet mesa
else:
  print("OK then.")

In [ ]:
import mesa
import numpy as np
import matplotlib.pyplot as plt

### Random Generator

Throughout the workshop we will use some random numbers.
* Mesa has its own random number generator. To avoid dealing with an extra new thing today, we'll use Numpy's.
* As per Numpy's suggestion, we will create a random number generator.

In [ ]:
rng = np.random.default_rng()

### Quick Class review

Let's review quickly how to declare classes in python, and how to create instances of your class.

In [ ]:
class  Greeter:
  def __init__(self, some_name):
    self.name = some_name

  def greet(self):
    print(f"Hello world. My name is {self.name}.")

In [ ]:
alice = Greeter("Alice")

In [ ]:
alice.greet()

This is how you define a subclass:

In [ ]:
class EarlyRiser(Greeter):
  preference = "Morning"
  def __init__(self, some_name, some_time):
    super().__init__(some_name)
    self.wakeup_time = some_time

  def awake_time(self):
    print(f"I'm usually awake by {self.wakeup_time}")

In [ ]:
alice2 = EarlyRiser("Alice 2", "5 AM")

In [ ]:
alice2.name

In [ ]:
alice2.greet()

In [ ]:
alice2.awake_time()

In [ ]:
alice2.preference

Notice that while we didn't explicitly indicate the `name` aatribute and the `greet()` method in `EarlyRiser`, alice2 inherited those from `Greeter`.

#### **EXERCISE**
Create a `GrumpyRiser` greeter with a method `coffee_intake()` which reports how much coffee your instance drinks to get energized. `GrumpyRiser` should be a subclass of `Greeter`.

<br><br><br><br><br><br>

## Mesa Classes

Mesa has some pre-established classes with handy methods. The must important ones are:
*   The Model class
*   The Agent class
*   Space related classes
*   Time related classes
*   Data Collection related classes

Usually, you will use the time, space, and data collection classes as they come. However, you will write your own model and agent classes that inherit from the base Mesa model and agent classes.

### Model Classes

Let's create a subclass of `mesa.Model`.

In [ ]:
class GreetModelV0(mesa.Model):          # <-- Indicates the parent class
  def __init__(self, model_name):
    super().__init__()                # <-- Initializes using the parent class
    self.name = model_name

**Note:** `mesa.Model()` does not require any initial arguments. You can create a model without a model_name. `TestModel` adds the required model_name parameter.

In [ ]:
model_0 = GreetModelV0("First Mesa Model")

In [ ]:
model_0.name

Cool! But our model doesn't doa nything interesting. For that, we'll have to create agents.

### Agent classes in Mesa

In [ ]:
class GreeterV1(mesa.Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)      # <-- Note mesa.Agent requires two arguments: unique_id and a mesa.Model

  def greet(self):
    print(f"Hello world. My name is {self.unique_id}.")

Note: Here, `mesa.Agent()` does require two parameters: a `unique_id` (to identify agents), and a `model` (the "system" agents are part of).

In [ ]:
mesa_greeter = GreeterV1(1, model_0)

In [ ]:
mesa_greeter.unique_id

In [ ]:
mesa_greeter.model

### Creating Agents inside the Model

The idea behind Mesa is that we should only worry about bird's eye view.

Hence, instead of creating agents ourselves, and individually asking them to perform tasks, we use the model for that.

In [ ]:
class GreetModelV1(mesa.Model):
  def __init__(self, N):
    super().__init__()
    self.number_agents = N
    self.greeters = []
    for n in range(self.number_agents):
      g = GreeterV1(unique_id = n,model = self) # <-- Note the input for require model argument
      self.greeters.append(g)

In [ ]:
N = 10
model_1 = GreetModelV1(N)

In [ ]:
model_1.greeters

**Note:**

Inside GreetModelV1, we explicitly wrote the agent class. This may become cumbersome or limiting. From now on I will use the agent class as an input for our models. For example:

In [ ]:
class GreetModelV1b(mesa.Model):
  def __init__(self, N, agent_class):
    super().__init__()
    self.number_agents = N
    self.greeters = []
    for n in range(self.number_agents):
      g = agent_class(n,self)
      self.greeters.append(g)

In [ ]:
N = 3
model_1b = GreetModelV1b(N, GreeterV1) # <-- input a reference to the class GreeterV1
model_1b.greeters

OK cool. But...

Making a list of agents is already looking like a lot of micro-managing. We don't want this.

This is were Mesa becomes quite handy! It manages our agents through a **scheduler**.

## The Scheduler

In a usual run of an agent-based model, we see two units of time:
*  Time at the agent level
*  Time at the model level

At each time step at the model level, a subset (or all) of the agents move and perform some actions.

At each time step at the agent level, only one agent moves and initiates actions.

Both Mesa models and agents use a `step()` method, indicating what to do at a given step. The scheduler manages who goes next.

In [ ]:
class GreeterV2(mesa.Agent):
  def __init__(self, unique_id, model, some_name):    # <-- Adding a name argument for fun.
    super().__init__(unique_id, model)
    self.name = some_name

  def greet(self):
    print(f"Hello world. My name is {self.name}.")

  def step(self):                 # <-- Adding step() method
    self.greet()

In [ ]:
class GreetModelV2(mesa.Model):
  def __init__(self, N, agent_class):
    super().__init__()
    self.number_agents = N

    # Create a scheduler!
    self.schedule = mesa.time.RandomActivation(self)                    # <-- Schedule

    # When you create agents, add them to the scheduler
    for n in range(self.number_agents):
      g = agent_class(n,self, f"{n}")
      self.schedule.add(g)      # <--  Add to schedule

  def step(self):
    self.schedule.step()        # <--  Tell schedule to step

**Note:**

There are different types of schedulers. We will use `RandomActivation`, which calls agents at a random order at each model level step.

In [ ]:
N = 10
model_2 = GreetModelV2(N, GreeterV2) # <-- You may get a warning when first running this about AgentSet. Ignore it.

In [ ]:
model_2.step()

Note how for 1 step of the model, each agent took their own agent-level step!

### RECAP

Agents
*   Need to be connected to a given model
*   Need a step() method

Models
* In charge of creating agents
* In charge of creating a schedule
* In charge of adding agents to the schedule
* Need a step() method calling the schedule's own step.



#### **EXERCISE**

Create a Mesa model `MoveModel` and agents in this model `Movers` that move in the unit square at random at each step and output their coordinates. Use a scheduler. Run it for 3 agents and a few steps.

Hint: you can select a random coordinate in the unit square with the following code: `(cx, cy) = rng.random(2)`

<br><br><br><br><br><br>

## Space

Another handy Mesa benefit is the pre-made space classes. For now it can only deal with square grids and 2-d continuous space consistently. Seems like other grid shapes (hexagonal, networks, etc.) are in development.

### Creating space and placing agents in space

In [ ]:
class GreetModelV3(mesa.Model):
  def __init__(self, N, agent_class, grid_side):
    super().__init__()
    self.number_agents = N
    self.schedule = mesa.time.RandomActivation(self)

    # Create grid!
    self.grid = mesa.space.MultiGrid(width = grid_side, height = grid_side, torus = True)

    # When you create agents, add them to the scheduler AND to the grid
    for n in range(self.number_agents):
      g = agent_class(n,self,f"{n}")
      self.schedule.add(g)

      [i,j] = rng.integers(grid_side,size=2)
      self.grid.place_agent(g, (i,j))        # <-- Add agent to the grid

  def step(self):
    self.schedule.step()

We will get back to the code above, but first let's visualize our grid.

In [ ]:
model_3 = GreetModelV3(N = 10, agent_class = GreeterV2, grid_side =3)

In [ ]:
model_3.grid

Let's make a table with agent number per grid cell

In [ ]:
# Recover grid dimensions
grid_side = model_3.grid.width

# Create numpy 2d array with grid dimensions
count_table = np.zeros((grid_side,grid_side))

# Get contents of each cell
for cell_content, (i,j) in model_3.grid.coord_iter():         # <--- coordinates iterator!
  n_agents = len(cell_content)
  count_table[i][j] = n_agents

In [ ]:
count_table

In [ ]:
plt.imshow(count_table, cmap = "Purples")

**MultiGrid:** The "multi" comes from the fact that many agents are allowed to live inside one cell. You could enforce one agent per cell, but we won't do that here.

**Grid Dimensions:** Notce MultGrid requires obth width and height. Since we are only interested in a square grid now we'll use a single grid_side parameter.

**Torus:** A torus is just a donut. This is the space you get when you have a square and allow the left side to "teleport" to the right side and the bottom to teleport to the top. Similar to the game Pac-Man. If set to False the agent just goes out of bounds (I don't think they have a way to enforce hard boundaries yet).

**Note on coordiantes:** Notice I've labeled the random coordiantes $i$ and $j$. That's because grids work like matrices, the first coordinate is the row (starting from the top) and the second the column (starting from the left). We are used to index matrices with $i$ and $j$. If we were dealing with continuous space, I would have labeled them $x$ and $y$.



### Moving agents in space

Often, agents interact only with those who are close to them. This is, live in the same neighborhood. For grids, there are two common types of neighborhoods:

*   Moore: Includes all 8 surrounding cells.
*   Von Neumann: Excludes diagonal neighboring cells.

We will use the Moore neighborhood.

Let's move our agents at each step to one of their neighboring cells:

In [ ]:
class GreeterV3(mesa.Agent):
  def __init__(self, unique_id, model, some_name):
    super().__init__(unique_id, model)
    self.name = some_name

  def step(self):
    self.move()
    self.greet()

  def greet(self):
    print(f"Hello world. My name is {self.name}. I am at {self.pos}")

  def move(self):         # <--- Adding move() method
    neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)       # <-- get a moore neighborhood
    [ci,cj] = rng.choice(neighborhood)
    self.model.grid.move_agent(self, (ci,cj))       # <-- move to new position in the grid!

In [ ]:
model_4 = GreetModelV3(10, GreeterV3, 3)

In [ ]:
model_4.step()

### Recap
Mesa provides us with two types of space:

*   Grids
*   Flat Continuous (2-d)

Don't forget to add your agents to space when you create them.

Notice that Mesa has some limitations, for example, it doesn't have the sphere as a possible space, but it would be the natural space for many real-life applications.

## Interacting Agents

We are finally ready to make our agents interact! We will do this by obtaining the cell contents of the grid.

Let's do the following: Every time they are in a cell with other agents, the agent adds them to a list of acquaintances. For now, we will re-count agents they already met.

In [ ]:
class UltimateGreeter(mesa.Agent):
  def __init__(self, unique_id, model, some_name):
    super().__init__(unique_id, model)
    self.name = some_name

    # -- New property --
    self.acquaintances = []
    self.n_acqs = len(self.acquaintances)

  def step(self):
    self.move()
    self.greet_neighbor()
    self.greet()

  def greet(self):
    print(f"Hello world. My name is {self.name}. I am at {self.pos}. Today I met: {self.acquaintances}.")
    return ;

  def move(self):
    neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
    [ci,cj] = rng.choice(neighborhood)
    self.model.grid.move_agent(self, (ci,cj))

  # -- New method --

  def greet_neighbor(self):
    neighbors = self.model.grid.get_cell_list_contents(self.pos)      # Obtain contents of cell
    for neigh in neighbors:
      if neigh.unique_id != self.unique_id:
        self.acquaintances.append(neigh.unique_id)
    self.n_acqs = len(self.acquaintances)



In [ ]:
model_5 = GreetModelV3(10, UltimateGreeter, 3)

In [ ]:
model_5.step()

<h4>
<font color = "red">
<b>NOTE</b>
</font>
</h4>

Next, we will take many model level steps, hence, go to your `UltimateGreeter` class and comment out the `greet()` method within `step()` or the `print()` statement.

## Observer

Finally, Mesa has observer classes which they call `DataCollector`. The data collector has two types of reporters:
* Model level reporters: report on properties of the model at each step. Can also report on model-wide functions (statistics, etc.).
* Agent level reporters: report on individual agent properties or functions of agents.

We will only see agent reporters here.

In [ ]:
class UltimateGreetingModel(mesa.Model):
  def __init__(self, N, agent_class, grid_side):
    super().__init__()
    self.number_agents = N
    self.schedule = mesa.time.RandomActivation(self)
    self.grid = mesa.space.MultiGrid(width = grid_side, height = grid_side, torus = True)
    self.initialize_agents(agent_class) # <-- added this to clean things up

    # -- Create Observer --
    self.data_collector = mesa.DataCollector(
        agent_reporters={"Acquaintances": "n_acqs"}
    )

  def step(self):
    self.data_collector.collect(self)       # <-- Call the observer!!
    self.schedule.step()

  def initialize_agents(self, agent_class):
      for n in range(self.number_agents):
        g = agent_class(unique_id = n,model = self, some_name = f"{n}")
        self.schedule.add(g)
        [ci,cj] = rng.integers(grid_side,size=2)
        self.grid.place_agent(g, (ci,cj))

The reporter requires a dictionary as argument. Your dictionary can have many key-value pairs. The key is just the name you give to a specific report. The value is the important part here. It can be the name of an agent attribute (as in our case). It can also be a functions you have declared or lambda functions. See Mesa general documentation and also the comments [here](https://github.com/projectmesa/mesa/blob/main/mesa/datacollection.py).

In [ ]:
model_6 = UltimateGreetingModel(3, UltimateGreeter, 3)

The data collector returns the data in the form of a pandas data frame:

In [ ]:
model_6.data_collector.get_agent_vars_dataframe()

In [ ]:
model_6.step()

In [ ]:
model_6.data_collector.get_agent_vars_dataframe()

Let's run the model for 50 steps and then plot the number of acquaintances per agent!

In [ ]:
model_6b = UltimateGreetingModel(3, UltimateGreeter, 3)

for k in range(50):
  model_6b.step()

df1 = model_6b.data_collector.get_agent_vars_dataframe()
df1.head(5)

In [ ]:
df1.reset_index().pivot(index = "Step", columns = "AgentID", values = "Acquaintances").plot(ylabel = "Number of Acquaintances")
# my pandas know-how is not the best, feel free to improve the line above.

#### **EXERCISE**
Below I've pasted copies of `UltimateGreeter` and `UltimateGreetingModel`. Modify them to achieve the following:

1. Agents don't double count acquaintances.

2. Add a "Percentage Met" report on the percentage of total agents an agent has met. You can do this by creating a property of the agent, like `n_acqs` above, or through a lambda function as the value in the dictionary (the input to the lambda function represents the agent).

In [ ]:
class UltimateGreeterV2(mesa.Agent):
  def __init__(self, unique_id, model, some_name):
    super().__init__(unique_id, model)
    self.name = some_name
    self.acquaintances = []
    self.n_acqs = len(self.acquaintances)


  def step(self):
    self.move()
    self.greet_neighbor()
    self.get_percentage()

  def move(self):
    neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
    [ci,cj] = rng.choice(neighborhood)
    self.model.grid.move_agent(self, (ci,cj))

  def greet_neighbor(self):
    neighbors = self.model.grid.get_cell_list_contents(self.pos)      # Obtain contents of cell
    for neigh in neighbors:
      if neigh.unique_id != self.unique_id:
        self.acquaintances.append(neigh.unique_id)
    self.n_acqs = len(self.acquaintances)

In [ ]:
class UltimateGreetingModelV2(mesa.Model):
  def __init__(self, N, agent_class, grid_side):
    super().__init__()
    self.number_agents = N
    self.schedule = mesa.time.RandomActivation(self)
    self.grid = mesa.space.MultiGrid(width = grid_side, height = grid_side, torus = True)
    self.initialize_agents(agent_class) # <-- added this to clean things up

    # -- Create Observer --
    self.data_collector = mesa.DataCollector(
        agent_reporters={"Acquaintances": "n_acqs"}
    )

  def step(self):
    self.data_collector.collect(self)       # <-- Call the observer!!
    self.schedule.step()

  def initialize_agents(self, agent_class):
      for n in range(self.number_agents):
        g = agent_class(unique_id = n,model = self, some_name = f"{n}")
        self.schedule.add(g)
        [ci,cj] = rng.integers(grid_side,size=2)
        self.grid.place_agent(g, (ci,cj))

In [ ]:
model_7 = UltimateGreetingModelV2(3, UltimateGreeterV2, 3)
for k in range(10):
  model_7.step()

df_ex = model_7.data_collector.get_agent_vars_dataframe()
df_ex.reset_index().pivot(index = "Step", columns = "AgentID", values = "Percentage Met").plot(ylabel = "Percentage of Agents Met")

<br><br><br><br><br><br>

## Parameter Sweep

Finally, Mesa comes in very handy with their `batch_run()` function. This is basically a parameter sweep. We won't have time for that in this workshop but I wanted to make you aware of it, since it is an integral part of simulation. You can learn more about it in Mesa's tutorial [here](https://mesa.readthedocs.io/en/stable/tutorials/intro_tutorial.html#batch-run).

In the slides for this workshop I have added a table illustrating how the output of a batch run would look.

## Conclusion

See last slide in workshop slides for recap of Mesa's classes and relationships.

And with that we come to and end. For further information, check out Mesa's own documentaion [here](https://mesa.readthedocs.io/en/stable/overview.html).

<br><br><br><br><br><br>

